In [1]:
import requests
import os
import pandas as pd
import sqlalchemy as sql
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData
import datetime as dt
import pyodbc
import produceQBD
path = os.path.join("PPMDBSQLite.db")
engine = create_engine("sqlite+pysqlite:///"+path)
inspector = inspect(engine)
Base = automap_base()
Base.prepare(engine, reflect=True) 
session = Session(engine)


In [3]:
nameType = {}
for table in Base.classes.keys():
    nameType[table] = {}
    for col in inspector.get_columns(table):
        typeT = col["type"]
        nameType[table][col["name"]] = str(col["type"])

In [4]:
def primaveraColDataTypeConverter(colType:str) -> str: 
    finalString = ""
    if "TEXT" in colType:
        finalString = "String"
    elif "INTEGER" in colType:
        finalString = "int"
    elif "DATETIME" in colType:
        finalString = "DateTime"
    elif "BLOB" in colType:
        finalString = "byte[]"
    elif "decimal" in colType.lower():
        finalString = "decimal"
    elif "REAL" in colType:
        finalString = "float"
    return finalString

def getCharacterNumFromColType(colType:str) -> str: 
    split1 = colType.split("(")[1]
    numberOfCharacters = split1.split(")")[0]
    return f"{numberOfCharacters}"
    
def buildClassString(tableName:str, colDict:dict) -> str: 
    resultString = "\t" + "public class " + tableName+"{\n\t\t"
    resultString += buildColumnFields(colDict)
    return resultString

def buildColumnFields(colDict:dict) -> str: 
    result =""
    for colName,colType in colDict.items():
        result += "public " + primaveraColDataTypeConverter(colType)+" "+colName + " {get; set;}\n\t\t"
    return result

def buildDBContextBody(tableName:str) -> str:
    return f"public DbSet<{tableName}> {tableName} " + "{get; set;}\n\t\t"

def addContextTail(contextStr:str, nameType:dict, Base, inspector) -> str:
    result = contextStr + "        protected override void OnModelCreating(ModelBuilder modelBuilder)\n\t\t\
        {\n\t\t\t\
            base.OnModelCreating(modelBuilder);\n\t\t\t"
    for table in nameType.keys():
        result += f"modelBuild.Entity<{table}>.HasNoKey();\n\t\t\t"
    result += "}}}"
    return result

def writeFile(contents, filename):
    f = open(filename, "w")
    f.write(contents)
    f.close()


In [5]:
namespaceHead = "using System;\n\
using System.Collections.Generic;\n\
using System.Linq;\n\
using System.Threading.Tasks;\n\
using System.ComponentModel.DataAnnotations;\n\
using System.ComponentModel.DataAnnotations.Schema;\n\
using Newtonsoft.Json;\n\
\n\
namespace UNKNOWNSPACE.Models\n\
{\n"

dbContextNamespaceHead = "using Microsoft.EntityFrameworkCore;\n\
using Microsoft.EntityFrameworkCore.Diagnostics;\n\
\n\
namespace UNKNOWNSPACE.Data\n\
{\n\t\
    public class ApplicationDbContext\n\t\
    {\n\t\t\
        public ApplicationDbContext(DbContextOptions<ApplicationDbContext> options)\n\t\t\t\
            : base(options)\n\t\t\
        {\n\
\n\t\t\
        }"
result = {}
result["dbContext"] = f"{dbContextNamespaceHead}\n\t\t"
for (table, colDict) in nameType.items():
    result[table] = {}
    result[table] = namespaceHead+buildClassString(table,colDict)+"}}"
    result["dbContext"] +=  buildDBContextBody(table)
result["dbContext"] = result["dbContext"][:-1]
result["dbContext"] = addContextTail(result["dbContext"], nameType, Base, inspector)
for k, v in result.items():
    if k == "dbContext":
        writeFile(v, "Data/Context/dbContext.cs")
    else:
        writeFile(v, f"Data/Models/{k}.cs")


In [2]:
produceQBD.main("sqlite+pysqlite:///"+path, "p6FromNB", True)


In [59]:
temp=[[{col:table} for col in inspector.get_pk_constraint(table)['constrained_columns']] for table in Base.classes.keys()]
result = []
for x in temp:
    [result.append(y) for y in x]
pks = {}
for i in result:
    for k,v in i.items():
        pks[k] = v
    

In [58]:
pks = produceQBD.createPKTableNameDictionary(Base, inspector)
fks = {}
copy = {}
for table in Base.classes.keys():
    columns = inspector.get_columns(table)
    for col in columns:
        if "_ID" in col['name']:
            if col['name'] in pks.keys():
                try:
                    fks[table].append( col['name'] +" FK - " + pks[col['name']]+f".{col['name']}")
                    copy[table].append( col['name'] )
                except:
                    fks[table] = [col['name'] +" FK - " + pks[col['name']]+f".{col['name']}"]
                    copy[table] = [col['name']]

In [57]:
dbDesign = ""
for tableName in Base.classes.keys():
    dbDesign += tableName
    dbDesign += "\n\n---\n\n"
    pk = inspector.get_pk_constraint(tableName)['constrained_columns']
    if tableName in fks.keys():
        foreignKeyStrings = fks[tableName]
        foreignKeyValues = [x.split(" ")[0] for x in foreignKeyStrings]
    else:
        foreignKeyStrings = [""]
        foreignKeyValues=[""]
    for column in inspector.get_columns(tableName):
        if column['name'] in pk:
            dbDesign += column['name'] +" PK"
        elif column['name'] in foreignKeyValues:
            dbDesign += foreignKeyStrings[foreignKeyValues.index(column['name'])]
        else:
            dbDesign += column['name']
        dbDesign+= "\n\n"
    dbDesign+= "\n\n"


            

In [71]:
filename = "testDynamicFK"
f = open(filename+ ".txt", "w")
f.write(dbDesign)
f.close()

In [55]:
projTable = Base.classes.PROJECT
taskRsrcTable = Base.classes.TASKRSRC
hrsID = session.query(projTable.PROJ_ID).filter("HRSA" == projTable.PROJ_SHORT_NAME).first()
a = session.query(func.sum(taskRsrcTable.TARGET_COST)).filter(taskRsrcTable.PROJ_ID == "370").all()
